## 流程示意
- 生成汇编指令
- 生成机器码
- 加载Logisim组件
- 开始自动化测试
  

### Assembly!

In [2]:
import re
import random
import sys
import os
import shutil
import threading
from time import sleep
import time

In [3]:
"""
    Generate Assemble with:
    ori, lui, 
    add, sub, lw, sw,
    beq
"""

# Reg Index, Just for better look
regs = [f"{i}" for i in range(1, 32)] 

R_type = ['add', 'sub']
I_type = ['ori', 'lui', 'lw', 'sw']
J_type = ['j', 'jal', 'jr']

def R(rs, rt, rd):
    op = random.choice(R_type)
    return f"{op} ${rd}, ${rs}, ${rt}"

def I(rs, rt, imm):
    op = random.choice(I_type)
    if op == 'beq':
        return f"{op} ${rs}, ${rt}, {imm}"
    elif op in ['lw', 'sw']:
        return f"{op} ${rt}, {imm}(${rs})"  # lw $rt, offset($rs)
    else:
        return f"{op} ${rt}, {imm}"
        
# def J(imm, random_code):

def generate_random_code(num_instructions=10):
    """
        I don't think 10 ins is enough but just use it?
    """
    code = []
    for _ in range(num_instructions):
        if random.random() < 0.4:
            rs, rt, rd = random.sample(regs, 3)
            code.append(R(rs, rt, rd))
        else:
            rs = random.choice(regs)
            rt = random.choice(regs)
            if random.choice(I_type) == 'beq':
                imm = str(random.randint(-128, 127) * 4)
            else:
                imm = str(random.randint(0, 65535)) if random.random() < 0.8 else hex(random.randint(0, 65535))
            code.append(I(rs, rt, imm))
    return "\n".join(code)


In [4]:
"""
    For Generating Related Assembly.txt File
"""

def write_to_file(filename="AssemblyP3.asm"):
    print("Generating Assembly!")
    code = generate_random_code(100)
    with open(filename, 'w') as f:
        sleep(0.5)
        print("File Open!")
        f.write(code)
        sleep(0.5)
    print(f"Code Generated in {filename} Successfully!")
# print(code)
filename="AssemblyP3.asm"
write_to_file()

Generating Assembly!
File Open!
Code Generated in AssemblyP3.asm Successfully!


In [5]:
cmd = (
    f'java -jar "C:\\Users\\Harudog\\Desktop\\Use\\Working Area\\Tools\\Co\\Mars4_5.jar" '
    f'"{filename}" '
    f'p nc mc CompactDataAtZero a dump .text HexText command.txt'
)

print("Running MARS...")
print(cmd)
sleep(0.5)
print("-" * 60)

result = os.system(cmd)

if result == 0 and os.path.exists("command.txt"):
    print("Well Done！command.txt Generated")
    with open("command.txt", 'r') as f:
        content = f.read().strip()
    with open("command.txt", 'w') as f:
        f.write("v2.0 raw\n")
        f.write(content + "\n")
else:
    print("WTF, Errors happened!!!")

Running MARS...
java -jar "C:\Users\Harudog\Desktop\Use\Working Area\Tools\Co\Mars4_5.jar" "AssemblyP3.asm" p nc mc CompactDataAtZero a dump .text HexText command.txt
------------------------------------------------------------
Well Done！command.txt Generated


In [15]:
import os, re, subprocess

CPU      = r"C:\Users\Harudog\Desktop\Use\Working Area\Src Code\coCode\2025\coHw\p3\cpu_single.circ"
PARTNER  = r"C:\Users\Harudog\Desktop\Use\Working Area\Src Code\coCode\2025\coHw\p3\cpu_single.circ"
LOGISIM  = r"C:\Users\Harudog\Desktop\Use\Working Area\Tools\Co\logisim-win-2.7.1.exe"
TXT      = "command.txt"
RESULT   = "CPU_RESULT.txt"
RESULT_PARTNER   = "CPU_RESULT_P.txt"

code = open(TXT, encoding="utf-8").read().strip()

def inject(circ):
    txt = open(circ, encoding="utf-8").read()
    txt = re.sub(r'(addr/data: 10 32\n).*?(</a>)',
                 r'\1' + code + r'\n\2',
                 txt, count=1, flags=re.S)
    open(circ, "w", encoding="utf-8").write(txt)

# CPU = input("Please input the absolute path of your cpu")
# PARTNER = input("Please input the absolute path of your partner's cpu")

inject(CPU)
inject(PARTNER) 

spinner_running = True
process = None

def spinner():
    """Waiting and Spinning"""
    spin_chars = ['\\', '|', '/', '-']
    i = 0
    while spinner_running:
        sys.stdout.write(f'\rRunning... {spin_chars[i]}')
        sys.stdout.flush()
        i = (i + 1) % len(spin_chars)
        time.sleep(0.1)
    sys.stdout.write('\r' + ' ' * 20 + '\r')
    sys.stdout.flush()


confirm = input("You sure to start?")
if (confirm.strip().lower() == "y" or confirm.strip().lower() == "yes"):
    try:
        print("Running Logisim ...")
        spinner_thread = threading.Thread(target=spinner, daemon=True)
        spinner_thread.start()
        
        process = subprocess.Popen([
            "java", "-jar", LOGISIM,
            CPU, "-tty", "table" 
        ], stdout=open(RESULT, "w", encoding="utf-8"))
        
        process.wait()

        spinner_running = True
        process = None
        
        print(f"\nWell Done! Finished writing {RESULT}")

        process = subprocess.Popen([
            "java", "-jar", LOGISIM,
            PARTNER, "-tty", "table" 
        ], stdout=open(RESULT_PARTNER, "w", encoding="utf-8"))

        process.wait()
        
        spinner_running = False
        spinner_thread.join(timeout=1)
        
        print(f"\nWell Done! Finished writing {RESULT_PARTNER}")
        
    except KeyboardInterrupt:
        print("\nInterrupting...")
        spinner_running = False
        if process:
            process.terminate()
            try:
                process.wait(timeout=5)
            except subprocess.TimeoutExpired:
                process.kill()
        print("Ok, done interrupted")
else:
    print("ok, we got it.")

You sure to start? yes


Running Logisim ...
Running... /
Well Done! Finished writing CPU_RESULT.txt
                    
Well Done! Finished writing CPU_RESULT_P.txt


In [16]:
import filecmp
def simple_cmp(file1, file2):
    if filecmp.cmp(file1, file2, shallow=False):
        print(True)
    else:
        print(False)
simple_cmp(RESULT, RESULT_PARTNER)

True
